In [23]:
import torch
import torch.nn as nn
import torch.nn.functional
import torch_geometric.nn 
from torch_geometric.utils import from_networkx
from torch_geometric.data import Data
import pandas as pd
import networkx as nx
import numpy as np
import tqdm
import random
from torch_geometric.nn import GATConv


In [11]:
pkl_graphs = pd.read_pickle('/Users/MathildeStouby/Desktop/P5 GitHub/5-semester/Momentum graphs.pkl')

In [12]:
unique_nodes = []
for graph in pkl_graphs.values():
    temp = [node for node in graph.nodes() if node not in unique_nodes]
    unique_nodes.extend(temp)

In [13]:
idx_to_pos = dict(enumerate(unique_nodes))
pos_to_idx = {pos : idx for idx, pos in idx_to_pos.items()}

In [14]:
pyg_data = []
thors_kamp = []


for idx, graph in pkl_graphs.items():

    closeness = nx.closeness_centrality(graph)
    betweenness = nx.closeness_centrality(graph)
    pagerank = nx.pagerank(graph, weight='weight')
    centrality_list = [closeness, betweenness, pagerank] 

    #adj_dict = nx.to_dict_of_dicts(graph)

    for node in list(graph.nodes()):
        #adj_vect = np.zeros((len(unique_nodes)))
        #players = adj_dict[node]
        #for key, value in players.items():
            #adj_vect[pos_to_idx[key]] = value['weight']
        #adj_vect = torch.from_numpy(adj_vect).float()
        centrality_vect = []
        for measure in centrality_list:
            if node in list(measure.keys()):
                centrality_vect.append(measure[node])
            else:
                centrality_vect.append(0)
        centrality_vect = torch.Tensor(centrality_vect).float()        
        #graph.nodes[node]['x'] = torch.cat((adj_vect, centrality_vect), -1)
        graph.nodes[node]['x'] = centrality_vect

   
    for node in unique_nodes:
        if node not in graph.nodes:
            graph.add_node(node) 
            #graph.nodes[node]['x'] = torch.from_numpy(np.zeros((len(unique_nodes)+3))).float()
            graph.nodes[node]['x'] = torch.from_numpy(np.zeros(3)).float()  
            

    data = from_networkx(graph)

    try:
        data['momentum']
        if idx.startswith('3895275'):
            thors_kamp.append(data)
        else:
            pyg_data.append(data)
    except:
        print(data)

Data(x=[23, 3], edge_index=[2, 22], weight=[22])


In [15]:
train_idx = random.sample(range(len(pyg_data)), int(len(pyg_data) * 0.8))
test_idx = [i for i in range(len(pyg_data)) if i not in train_idx]

In [21]:
class GAT(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_heads):
        super(GAT, self).__init__()
        self.layer1= GATConv(input_dim, hidden_dim, heads=num_heads, edge_dim=1)
        self.layer2= GATConv(hidden_dim * num_heads, hidden_dim, heads=num_heads, edge_dim=1)
        self.layer3 = GATConv(hidden_dim * num_heads, output_dim, heads=1, edge_dim=1, concat=False)
        self.activation_function = nn.ReLU()
     

    def forward(self, input, edge_index, edge_attr):
        output = self.activation_function(self.layer1(input, edge_index, edge_attr))
        output = self.activation_function(self.layer2(output, edge_index, edge_attr))
        output = self.layer3(output, edge_index, edge_attr)
        output = torch.mean(output)
        return output

In [24]:
input_dim = 3
lr = 0.01

gat = GAT(input_dim = input_dim, hidden_dim = 11, output_dim = 1, num_heads = 6)
optimizer = torch.optim.SGD(gat.parameters(), lr=lr)
loss_fn = torch.nn.MSELoss()
epochs_num = 100

In [25]:
for epoch in tqdm.tqdm(range(epochs_num)):
    
    epoch_loss = 0
    for idx in tqdm.tqdm(range(len(train_idx))):
        
        input = pyg_data[idx].x
        edge_idx = pyg_data[idx].edge_index
        edge_attr = pyg_data[idx].weight
        label = pyg_data[idx]['momentum']
    
        # Forward pass
        optimizer.zero_grad()
        
        output = gat(input, edge_idx, edge_attr)
        
        # Calculate loss
        loss = loss_fn(output, label)
        loss.backward()
        # optimizer.step()
        for p in gat.parameters():
            p.data.add_(p.grad.data, alpha=-lr)
        
        epoch_loss += loss.item()
    
    print(f"Epoch {epoch+1}, Loss: {epoch_loss/len(train_idx):.4f}") 
gat.eval() 

  1%|          | 1/100 [00:02<03:57,  2.40s/it]

Epoch 1, Loss: 0.0067


  2%|▏         | 2/100 [00:04<03:24,  2.09s/it]

Epoch 2, Loss: 0.0066


  3%|▎         | 3/100 [00:06<03:11,  1.97s/it]

...

 99%|█████████▉| 99/100 [03:15<00:01,  1.97s/it]

Epoch 99, Loss: 0.0064


100%|██████████| 100/100 [03:17<00:00,  1.98s/it]

Epoch 100, Loss: 0.0064


GAT(
  (layer1): GATConv(3, 11, heads=6)
  (layer2): GATConv(66, 11, heads=6)
  (layer3): GATConv(66, 1, heads=1)
  (activation_function): ReLU()
)

In [26]:
y_pred  = []
y_true = []

with torch.no_grad():
    for idx in test_idx:
        batch = torch.zeros(pyg_data[idx].x.size(0), dtype=torch.long)
        output = gat(pyg_data[idx].x, pyg_data[idx].edge_index, pyg_data[idx].edge_attr)
        y_pred.append(output.item())
        y_true.append(pyg_data[idx].momentum)
        
from sklearn.metrics import r2_score
r2_score(y_true, y_pred)

0.01789867722752314